In [6]:
# this specifically tests LIB-HSI VAL
import os, shutil, cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba, to_hex
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.colors as mcolors
from scipy.ndimage import zoom

In [7]:
#LIB Dataset labels
# data = [
#     ['classname_full', 'classname_short', 'color', 'class_number'],
#     ["miscellaneous", 'miscellaneous', 'orange', '0'],
#     ["vegetation", 'vegetation', 'pink', '1'],
#     ["glass window", 'glass window', 'blue', '2'],
#     ["brick surface", 'brick', 'red', '3'],
#     ["concrete surface", 'concrete surface', 'white', '4'],
#     ["concrete block wall", 'concrete blocks', 'yellow', '5'],
#     ["metal", 'metal', 'black', '6'],
#     ["door", 'door', 'purple', '7'],
#     ["timber", 'timber', 'green', '8'],
# ]

#facade datset labels
data = [['classname_full', 'classname_short', 'color','class_number'],
        ["glass surface", 'glass', 'blue',10],
        ["concrete surface", 'concrete', 'yellow',11],
        ["brick wall", 'brick', 'red',12],
       ["siding", 'siding', 'green',13],
#         ["roof tiles shingles", 'roof', 'orange',14],
        ["plaster facade surface", 'plaster', 'white',16],
        ["metal panels", 'metal', 'purple',17],
       ["door", 'door', 'brown',32],]


# Convert the list of lists to a DataFrame
label_df = pd.DataFrame(data[1:], columns=data[0])

# Convert the list of lists to a DataFrame
label_df = pd.DataFrame(data[1:], columns=data[0])

In [8]:
len(os.listdir('/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/MASKS'))

173

In [9]:
#functions that convert images to LIBHSI classes and vice versa
def create_image_from_numbers(number_array, color_mapping):
    height, width = number_array.shape[:2]
    img = np.zeros((height, width, 3), dtype=np.uint8)
    for i in range(height):
        for j in range(width):
            number = int(number_array[i, j])
            if number in color_mapping:
                colorvalue = np.asarray(to_rgba(mcolors.to_rgb(color_mapping[number]))[:3])
                img[i,j,0]=colorvalue[0]*255
                img[i,j,1]=colorvalue[1]*255
                img[i,j,2]=colorvalue[2]*255
    return img

def convert_image_to_numbers(image, color_mapping):
    height, width, _ = image.shape
    number_array = np.zeros((height, width), dtype=int)
    for i in range(height):
        for j in range(width):
            color = tuple(image[i, j, :3])
            number_array[i, j] = inverse_color_mapping.get((color), -1)
    return number_array


# Create color mapping and inverse color mapping
color_mapping = {int(row[3]): row[2] for row in data[1:]}
inverse_color_mapping = {(tuple((np.asarray(mcolors.to_rgba(mcolors.CSS4_COLORS[row[2]])[:3])*255).astype('int'))):int(row[3]) for row in data[1:]}



numbers_array = np.array([[0, 1, 2],
                          [3, 1, 4]])


image_from_numbers = create_image_from_numbers(numbers_array, color_mapping)
numbers_array_converted = convert_image_to_numbers(image_from_numbers, inverse_color_mapping)


In [10]:
def calculate_stats_per_class(result_image, ground_truth, class_number, presence_threshold=0.01):
    # Skip the "roof" class (assuming its class number is 14)
    #if class_number == 14:
        #return np.nan, np.nan, np.nan, np.nan


    ground_truth_presence = np.any(ground_truth == class_number)
    prediction_presence = np.any(result_image == class_number)

    # Calculate the presence ratio in the prediction
    prediction_ratio = np.sum(result_image == class_number) / result_image.size

    # Apply the threshold to consider minor presences as absences
    if not ground_truth_presence and (not prediction_presence or prediction_ratio < presence_threshold):
        print(f"Class {class_number} considered absent due to presence threshold.")
        return np.nan, np.nan, np.nan, np.nan


    mask_result = (result_image == class_number)
    mask_hsi = (ground_truth == class_number)
    intersection = np.logical_and(mask_result, mask_hsi).sum()
    union = np.logical_or(mask_result, mask_hsi).sum()

    # Correct weight calculation: ratio of class pixels to total class pixels in ground_truth
    total_class_pixels = ground_truth[ground_truth == class_number].size
    weight = mask_hsi.sum() / total_class_pixels if total_class_pixels != 0 else 0

    # Applying the weight to the IoU calculation
    iou = (intersection / union) * weight if union != 0 else 0

    # Calculate additional metrics
    true_positive = intersection
    false_positive = mask_result.sum() - true_positive
    false_negative = mask_hsi.sum() - true_positive
    true_negative = (~mask_result & ~mask_hsi).sum()

    # Calculate accuracy, precision, and recall
    total = true_positive + false_positive + false_negative + true_negative
    accuracy = (true_positive + true_negative) / total if total != 0 else 0
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) != 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) != 0 else 0

    return iou, accuracy, precision, recall

In [18]:
#NEW
NEWLABELS_PATH = '/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/MASKS/'
#PRED_PATH = '/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/OUTPUTS_CLIPSEG/CLIPSEG/'
#PRED_PATH = '/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/OUTPUTS/CLIPSEG/'
# NEWLABELS_PATH = '/home/klimenko/facade_materials/materials/train/labels_8/'
# PRED_PATH = '/home/klimenko/seg_materials/OUTPUTS/RUN7_LIB_HSI/CLIPSEG/'
PRED_PATH = '/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/OUTPUTS_CLIPSEG_FACTOR_2/CLIPSEG/'
#PRED_PATH = "/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/OUTPUTS_RAW_CLIPSEG/"
#PRED_PATH = '/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/OUTPUTS_RAW_CLIPSEG/'
#NEWLABELS_PATH = '/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/MASKS/'
#PRED_PATH = '/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/OUTPUTS_CLIPSEG/CLIPSEG/'
#PRED_PATH = '/home/klimenko/seg_materials/VAL_SEGFORMER/results/'
#RGB_PATH = '/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/RGB/'


count = 0
iou_total = []
accuracy_total = []
precision_total = []
recall_total = []
rgb_img_list = []
pred_img_list=[]
label_img_list = []

presence_threshold = 0.05#  # Adjust

for imagename in os.listdir(PRED_PATH):#['180__Iz-LiOvZi5rYWr6zZsvIZQ.npy']:
    #print(imagename)
    try:
        #print(imagename)
#         if imagename[:-4]+'.jpg' not in os.listdir('/home/klimenko/seg_materials/FINAL_DATASETS/BROOKLYN/MASKS/'):
#             raise Exception("NOT IN DSET")

        if imagename.endswith(".ipynb_checkpoints"):
            continue  # Skip non-relevant files

        count += 1
        pred_img = np.load(os.path.join(PRED_PATH, imagename))
        pred_img = pred_img.astype(np.uint8)
        label_img = cv2.imread(os.path.join(NEWLABELS_PATH, imagename[:-4] + '.jpg'), cv2.IMREAD_GRAYSCALE)
        truth = label_img ==8
        print(np.sum(truth))
        truth2 = pred_img ==8
        print(np.sum(truth2))
        print()
        pred_img = cv2.resize(pred_img, (label_img.shape[1], label_img.shape[0]))

        #print(f"Processing {imagename}: pred_img shape={pred_img.shape}, label_img shape={label_img.shape}")

        iou_scores = []
        accuracy_scores = []
        precision_scores = []
        recall_scores = []

        for class_number in [10, 11, 12, 13, 16, 17, 32]:
        #for class_number in [0, 1, 2, 3, 4, 5, 6, 7, 8]:

            # Calculate class presence ratio in prediction
            prediction_ratio = np.sum(pred_img == class_number) / pred_img.size
            label_ratio = np.sum(label_img == class_number) / pred_img.size

            # Determine if class should be considered absent
            if prediction_ratio < presence_threshold and label_ratio < presence_threshold:
                #print(f"Class {class_number} considered absent due to presence threshold.")
                iou = accuracy = precision = recall = np.nan
            else:
                iou, accuracy, precision, recall = calculate_stats_per_class(pred_img, label_img, class_number)

            iou_scores.append(iou)
            accuracy_scores.append(accuracy)
            precision_scores.append(precision)
            recall_scores.append(recall)

            #print(iou)

        iou_total.append(iou_scores)
        accuracy_total.append(accuracy_scores)
        precision_total.append(precision_scores)
        recall_total.append(recall_scores)
    except Exception as E:
        print(E, 'PASS')


124
0

385
0

111
0

747
0

802
0

124
0

834
0

58
0

182
0

197
0

165
0

49
0

0
0

'NoneType' object has no attribute 'shape' PASS
148
0

0
0

'NoneType' object has no attribute 'shape' PASS


[ WARN:0@249.696] global loadsave.cpp:248 findDecoder imread_('/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/MASKS/0__zAHgamHte7_XnCzprmZGsw.jpg'): can't open/read file: check file path/integrity
[ WARN:0@249.837] global loadsave.cpp:248 findDecoder imread_('/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/MASKS/0__N412ISFqyCYXkcpi8fKlVg.jpg'): can't open/read file: check file path/integrity


77
0

56
0

159
0

95
0

634
0

197
0

212
0

157
0

187
0

16
0

0
0

'NoneType' object has no attribute 'shape' PASS
24
0



[ WARN:0@250.827] global loadsave.cpp:248 findDecoder imread_('/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/MASKS/180__Iz-LiOvZi5rYWr6zZsvIZQ.jpg'): can't open/read file: check file path/integrity


1851
0

1490
0

23
0

177
0

80
0

1226
0

147
0

216
0

184
0

1870
0

36
0

90
0

739
0

218
0

1582
0

223
0

167
0

11
0

474
0

15
0

222
0

869
0

12
0

8
0

293
0

314
0

1180
0

658
0

209
0

11
0

82
0

13
0

2024
0

88
0

211
0

30
0

170
0

4
0

217
0

368
0

221
0

23
0

116
0

257
0

130
0

60
0

115
0

218
0

183
0

399
0

669
0

164
0

37
0

554
0

419
0

36
0

95
0

60
0

96
0

202
0

342
0

20
0

35
0

204
0

18
0

48
0

147
0

230
0

71
0

638
0

38
0

0
0

'NoneType' object has no attribute 'shape' PASS
133
0

166
0



...

67
0

0
0

'NoneType' object has no attribute 'shape' PASS
48
0



[ WARN:0@261.111] global loadsave.cpp:248 findDecoder imread_('/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/MASKS/0__oZv5IG66-SyDbwWUvGcV9g.jpg'): can't open/read file: check file path/integrity


368
0

341
0

117
0

0
0

'NoneType' object has no attribute 'shape' PASS
79
0

432
0



[ WARN:0@261.532] global loadsave.cpp:248 findDecoder imread_('/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/MASKS/0__Y79opRbZbCJNhLi4dgzUiw.jpg'): can't open/read file: check file path/integrity


978
0

310
0

173
0

27
0

1016
0

421
0

139
0

140
0

66
0

12
0

116
0

292
0

123
0

23
0

2447
0

1273
0



In [19]:
len(iou_total)

147

In [20]:
import numpy as np

# Convert lists of lists to NumPy arrays for easier manipulation
iou_array = np.array(iou_total)
accuracy_array = np.array(accuracy_total)
precision_array = np.array(precision_total)
recall_array = np.array(recall_total)

# Calculate mean metrics, ignoring NaNs
mean_iou = np.nanmean(iou_array)
mean_accuracy = np.nanmean(accuracy_array)
mean_precision = np.nanmean(precision_array)
mean_recall = np.nanmean(recall_array)

print("Average IoU across all images:", mean_iou)
print("Average accuracy across all images:", mean_accuracy)
print("Average precision across all images:", mean_precision)
print("Average recall across all images:", mean_recall)


Average IoU across all images: 0.3902633667399458
Average accuracy across all images: 0.8727999725053662
Average precision across all images: 0.5237373648445194
Average recall across all images: 0.5130989693845212


In [21]:
iou_total = np.asarray(iou_total)
accuracy_total = np.asarray(accuracy_total)
precision_total = np.asarray(precision_total)
recall_total = np.asarray(recall_total)

iou_means = np.nanmean(iou_total, axis=0)#[0:10]
accuracy_means = np.nanmean(accuracy_total, axis=0)#[0:10]
precision_means = np.nanmean(precision_total, axis=0)#[0:10]
recall_means = np.nanmean(recall_total, axis=0)#[0:10]

classnames = list(label_df['classname_short'])#[0:10]
df = pd.DataFrame([list(iou_means)], columns=classnames)
df

,glass,concrete,brick,siding,plaster,metal,door
0,0.625219,0.109005,0.482352,0.26513,0.229743,0.128984,0.47923


In [22]:
df = pd.DataFrame({
    'Class': classnames,
    'IOU': iou_means,
    'Accuracy': accuracy_means,
    'Precision': precision_means,
    'Recall': recall_means
})
numeric_columns = df.columns[1:]
#total_mean_row = df[numeric_columns].mean()
total_mean_row = df.mean(skipna=True, numeric_only=True)
total_mean_row['Class'] = 'TOTAL'  # Set the new class name
total_mean_row = pd.DataFrame(total_mean_row).T

# Append the total mean row to the DataFrame
result_df= df[df['Class'] != 'roof']
result_df = pd.concat([df, total_mean_row], axis=0)
result_df.to_csv('LIB-HSI-PATCH.csv')

def round_numeric_columns(x):
    if isinstance(x, (int, float, np.number)):
        return round(x, 3)
    return x

# Apply the function to the entire DataFrame
result_df = result_df.applymap(round_numeric_columns)
result_df = result_df.drop('Accuracy', axis=1)

/tmp/ipykernel_1143/4242024929.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  result_df = result_df.applymap(round_numeric_columns)


In [23]:
result_df # SEGFORMER

,Class,IOU,Precision,Recall
0,glass,0.625,0.735,0.818
1,concrete,0.109,0.348,0.126
2,brick,0.482,0.592,0.622
3,siding,0.265,0.388,0.354
4,plaster,0.230,0.375,0.314
5,metal,0.129,0.269,0.259
6,door,0.479,0.641,0.538
0,TOTAL,0.331,0.478,0.433


len(os.listdir('/home/klimenko/seg_materials/OUTPUTS/RUN7_LIB_HSI/COMBINED'))

In [17]:
len(iou_total)

116

In [136]:
print(len(os.listdir('/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/OUTPUTS_CLIPSEG/CLIPSEG/')))

144


In [75]:
print(len(os.listdir('/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/OUTPUTS_RAW_CLIPSEG')))

188


In [100]:
import shutil
import os

source_dir = '/home/klimenko/seg_materials/FINAL_DATASETS/BOSTON/RGB'  # Path to directory A
target_dir = '/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/RGB'  # Path to directory B

# Copy all files from source_dir to target_dir, overriding existing files
for file_name in os.listdir(source_dir):
    full_file_name = os.path.join(source_dir, file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, target_dir)

In [5]:
print(len(os.listdir('/home/klimenko/seg_materials/FINAL_DATASETS/COMBINED/OUTPUTS_CLIPSEG_FACTOR_1/CLIPSEG')))

120
